<a href="https://colab.research.google.com/github/Waynebadu/PL.repo/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

41371121H 科技117 黃昱瑋
[試算表連結](https://docs.google.com/spreadsheets/d/16Fudzxcy-m2brNoUM6AnELBWeYMm_cGi7TQnUqIUkKc/edit?gid=1597707981#gid=1597707981)

In [8]:
!pip install -q google-generativeai

In [9]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [10]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('agevjfgd')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [11]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/16Fudzxcy-m2brNoUM6AnELBWeYMm_cGi7TQnUqIUkKc/edit?gid=1597707981#gid=1597707981"
WORKSHEET_NAME = "工作表2"

REQUIRED_COLUMNS = ["日期", "科目", "作業成績"]

_auth_done = False
_gc = None
_ws = None

In [12]:
# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

In [13]:
def get_ai_summary(grades):
    """
    呼叫 Gemini 模型來生成成績摘要與常見迷思。
    """
    model = genai.GenerativeModel('gemini-1.5-flash')

    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成摘要... ---")
    try:
        response = model.generate_content(prompt_text)
        summary = response.text
        return summary
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

In [14]:
new_grades = get_user_grades()

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：100
請輸入 100 的成績：56
已記錄：日期: 2025-10-02, 科目: 100, 成績: 56

請輸入科目（或輸入 'q' 停止）：86
請輸入 86 的成績：77
已記錄：日期: 2025-10-02, 科目: 86, 成績: 77

請輸入科目（或輸入 'q' 停止）：89
請輸入 89 的成績：q
成績必須是數字。請重新輸入。
請輸入科目（或輸入 'q' 停止）：q


In [15]:
new_grades

[['2025-10-02', '100', 56], ['2025-10-02', '86', 77]]

In [16]:
get_ai_summary(new_grades)


--- 正在呼叫 AI 模型生成摘要... ---


呼叫 AI 時發生錯誤：404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.


'AI 摘要生成失敗。'

In [18]:
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)





        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()

--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：國文
請輸入 國文 的成績：80
已記錄：日期: 2025-10-02, 科目: 國文, 成績: 80

請輸入科目（或輸入 'q' 停止）：英文
請輸入 英文 的成績：99
已記錄：日期: 2025-10-02, 科目: 英文, 成績: 99

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---

--- 正在呼叫 AI 模型生成摘要... ---


呼叫 AI 時發生錯誤：404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

--- AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
AI 摘要生成失敗。
--------------------------------------------------
